In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

if os.path.exists('/Users'):
    data_path = '/Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/'
else:
    data_path = "D:\\work\\DataAnalysis\\study\\mnist"
    
mnist = input_data.read_data_sets(data_path, dtype=tf.uint8, one_hot=True)
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1]
num_examples = mnist.train.num_examples

filename = os.path.join('data', 'mnist.tfrecords')

writer = tf.python_io.TFRecordWriter(filename)

for index in range(num_examples):
    image_raw = images[index].tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels),
        'label': _int64_feature(np.argmax(labels[index])),
        'image_raw': _bytes_feature(image_raw)
    }))
    writer.write(example.SerializeToString())

writer.close()

Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf
import os

reader = tf.TFRecordReader()

filename = os.path.join('data', 'mnist.tfrecords')

filename_queue = tf.train.string_input_producer([filename])

_, serialized_example = reader.read(filename_queue)

features = tf.parse_single_example(serialized_example,
                                  features={
                                      'image_raw': tf.FixedLenFeature([], tf.string),
                                      'pixels': tf.FixedLenFeature([], tf.int64),
                                      'label': tf.FixedLenFeature([], tf.int64)
                                  })

images = tf.decode_raw(features['image_raw'], tf.uint8)
labels = tf.cast(features['label'], tf.int32)
pixels = tf.cast(features['pixels'], tf.int32)

sess = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])